In [31]:
import numpy as np
import pandas as pd
import os
import copy
import warnings
warnings.filterwarnings('ignore')

In [17]:
trainPath = '../data/eleme_round1_train' # 训练数据的路径
data_list = [ str(d) for d in range(20200201,20200230)] # 日期列表

In [18]:
# 数据读取方法
def getDF(path,fileName):
    df_list =[]
    for f in data_list:
        fn = fileName+'_'+f+'.txt'
        df = pd.read_csv(os.path.join(path,fileName,fn))
        df_list.append(df)
       
    return df_list

In [19]:
# 数据读取
train_action_list= getDF(trainPath,'action')
train_courier_list = getDF(trainPath,'courier')
train_distance_list = getDF(trainPath,'distance')
train_order_list = getDF(trainPath,'order')

In [21]:
# 列名字典
action_columns_name_dict={'courier_id':'骑手ID','wave_index':'波次','tracking_id':'订单号','courier_wave_start_lng':'骑手起始经度','courier_wave_start_lat':'骑手起始纬度','action_type':'动作类型','expect_time':'时间'}
courier_columns_name_dict={'courier_id':'骑手ID','level':'等级','speed':'速度','max_load':'最大负荷量'}
distance_columns_name_dict={'courier_id':'骑手ID','wave_index':'波次','tracking_id':'源订单号','source_type':'源类型','source_lng':'源经度','source_lat':'源纬度','target_tracking_id':'目标订单号','target_type':'目标类型','target_lng':'目标经度','target_lat':'目标纬度','grid_distance':'距离'}
order_columns_name_dict={'courier_id':'骑手ID','wave_index':'波次','tracking_id':'订单号','weather_grade':'天气','pick_lng':'商家纬度','pick_lat':'商家经度','deliver_lng':'客户纬度','deliver_lat':'客户经度','create_time':'创建时间','confirm_time':'确认时间','assigned_time':'通知时间','promise_deliver_time':'承诺送达时间','estimate_pick_time':'可以取单时间','aoi_id':'客户标识ID','shop_id':'商家标识ID'}

In [22]:
# 获取一个订单动作后对应的可选取单订单和可选送单订单的相关信息的方法
def OneOrderActionGetAllOption(OrderID,Action,ActionDF,OrderDF,DistanceDF,shuxing):
    action_time = ActionDF.loc[(ActionDF['订单号']==OrderID) & (ActionDF['动作类型']==Action),'时间'].values[0]
    order_timeBefor = ActionDF.loc[ActionDF['时间'] < action_time,:] # 小于当前动作完成前的所有订单及对应动作
    order_assign = pd.DataFrame(columns=['订单号','动作类型','时间','经度','纬度']) # 存放已发布，未提取
    order_pickup = pd.DataFrame(columns=['订单号','动作类型','时间','经度','纬度']) # 存放已取单，未送单
    last_action = order_timeBefor.iloc[-1,:] # 决策前的最后一个动作
    
    
    # 挑出
    for j in range(len(order_timeBefor)):
        orderid = order_timeBefor.iloc[j,0] 
        orderDF = order_timeBefor.loc[order_timeBefor['订单号']== orderid,:]
        if len(orderDF) == 1:
            order_assign=order_assign.append(ActionDF.loc[(ActionDF['订单号']==orderDF.iloc[0,0]) & (ActionDF['动作类型'] =='PICKUP'),:],ignore_index =True)
        elif len(orderDF) == 2:
            df1 = orderDF.iloc[1,:]
            order_pickup=order_pickup.append(ActionDF.loc[(ActionDF['订单号']==df1.iloc[0]) & (ActionDF['动作类型'] =='DELIVERY'),:],ignore_index =True)

    order_pickup.drop_duplicates(inplace=True)
    # 距离
    order_assign['距离'] =0 # 骑手距离商家
    order_pickup['距离'] =0 # 骑手距离客户
    #shuxing = courier.loc[courier['骑手ID']==peopleID,['等级','速度','最大负荷量']] # 骑手属性
    #distance_wave = distance.loc[(distance['骑手ID']==peopleID) & (distance['波次']==waveID)]
    for y in range(len(order_assign)):
        order_assign.iloc[y,5] = DistanceDF.loc[(DistanceDF['源订单号']==last_action['订单号']) & (DistanceDF['源类型']==last_action['动作类型'])&(DistanceDF['目标订单号']==order_assign.iloc[y,0])&(DistanceDF['目标类型']==order_assign.iloc[y,1]),'距离'].values[0]
    for z in range(len(order_pickup)):
        try:
            order_pickup.iloc[z,5] = DistanceDF.loc[(DistanceDF['源订单号']==last_action['订单号']) & (DistanceDF['源类型']==last_action['动作类型'])&(DistanceDF['目标订单号']==order_pickup.iloc[z,0])&(DistanceDF['目标类型']==order_pickup.iloc[z,1]),'距离'].values[0]
        except:
            print(z,len(order_pickup))
    
    # 时间
    order_assign['理论时间'] = ((order_assign.loc[:,'距离'] / shuxing.iloc[0,1])+last_action['时间']).apply(int)
    order_pickup['理论时间'] = ((order_pickup.loc[:,'距离'] / shuxing.iloc[0,1])+last_action['时间']).apply(int)

    # 补充订单可取时间，承诺送达时间,天气
    order_assign['订单分配时间'] =0
    order_assign['订单可取时间'] = 0
    order_assign['订单承诺送达时间'] = 0
    order_assign['天气'] = '1'   
    order_pickup['订单分配时间'] =0
    order_pickup['订单可取时间'] = 0
    order_pickup['订单承诺送达时间'] = 0
    order_pickup['天气'] = '1'
    
    for y in range(len(order_assign)):
        oneOrder = OrderDF.loc[OrderDF['订单号'] == order_assign.iloc[y,0],['通知时间','可以取单时间','承诺送达时间','天气']]
        
        order_assign.iloc[y,7] = oneOrder.iloc[0,0]
        order_assign.iloc[y,8] = oneOrder.iloc[0,1]
        order_assign.iloc[y,9] = oneOrder.iloc[0,2]
        order_assign.iloc[y,10] = oneOrder.iloc[0,3]
    
    for z in range(len(order_pickup)):
    
        oneOrder = OrderDF.loc[OrderDF['订单号'] == order_pickup.iloc[z,0],['通知时间','可以取单时间','承诺送达时间','天气']]
       
        order_pickup.iloc[z,7] = oneOrder.iloc[0,0]
        order_pickup.iloc[z,8] = oneOrder.iloc[0,1]
        order_pickup.iloc[z,9] = oneOrder.iloc[0,2]
        order_pickup.iloc[z,10] = oneOrder.iloc[0,3]
    
    order_assign.rename(columns={'订单号':'取单号'},inplace=True)
    order_pickup.rename(columns={'订单号':'送单号'},inplace=True)
    
    return order_assign,order_pickup,last_action
    
    
    
    

In [23]:
# 整理
def GetAllOptionOrder(orderSort,actionBig,order,distance,courier):
    pickupBig = pd.DataFrame(columns=['订单号','动作','动作时间','通知时间','可以取单时间','承诺送达时间','骑手上个动作时间','骑手经度','骑手纬度','骑手距离','骑手等级','骑手速度','最大负荷量','取单号','动作类型','时间','经度','纬度','距离','理论时间','订单分配时间','订单可取时间','订单承诺送达时间','天气'])
    deliveryBig = pd.DataFrame(columns=['订单号','动作','动作时间','通知时间','可以取单时间','承诺送达时间','骑手上个动作时间','骑手经度','骑手纬度','骑手距离','骑手等级','骑手速度','最大负荷量','送单号','动作类型','时间','经度','纬度','距离','理论时间','订单分配时间','订单可取时间','订单承诺送达时间','天气'])
    #print(actionBig)
    for i in range(len(orderSort)):
        orderID = orderSort.loc[i,'订单号']
        waveID = orderSort.loc[i,'波次']
        peopleID = orderSort.loc[i,'骑手ID']
        orderAction = orderSort.loc[i,'动作']
        action_wave = actionBig.loc[(actionBig['骑手ID']==peopleID) & (actionBig['波次']==waveID),['订单号','动作类型','时间','经度','纬度']] #这个波次的所有动作 
        order_wave =order.loc[(order['骑手ID']==peopleID) & (order['波次'] ==waveID),:]
        shuxing = courier.loc[courier['骑手ID']==peopleID,['等级','速度','最大负荷量']] # 骑手属性
        distance_wave = distance.loc[(distance['骑手ID']==peopleID)&(distance['波次'] == waveID)]
        pickup,delivery,last_action= OneOrderActionGetAllOption(orderID,orderAction,action_wave,order_wave,distance_wave,shuxing)
        
        pickup['订单号'] = orderID
        pickup['动作'] = orderAction
        pickup['动作时间'] =action_wave.loc[(action_wave['订单号']==orderID)&(action_wave['动作类型']==orderAction),'时间'].values[0]
        orderTime = order_wave.loc[order_wave['订单号']==orderID,['通知时间','可以取单时间','承诺送达时间']]
        pickup['通知时间'] =orderTime.iloc[0,0]
        pickup['可以取单时间'] =orderTime.iloc[0,1]
        pickup['承诺送达时间'] = orderTime.iloc[0,2]
        pickup['骑手上个动作时间'] = last_action['时间']
        pickup['骑手经度'] = last_action['经度']
        pickup['骑手纬度'] = last_action['纬度']
        pickup['骑手距离'] = distance_wave.loc[(distance_wave['源订单号']==last_action['订单号']) & (distance_wave['源类型']==last_action['动作类型'])&(distance_wave['目标订单号']==orderID)&(distance_wave['目标类型']==orderAction),'距离'].values[0]
        pickup['骑手等级'] =shuxing.iloc[0,0]
        pickup['骑手速度'] =shuxing.iloc[0,1]
        pickup['最大负荷量'] = shuxing.iloc[0,2]
        
        delivery['订单号'] = orderID
        delivery['动作'] = orderAction
        delivery['动作时间'] =action_wave.loc[(action_wave['订单号']==orderID)&(action_wave['动作类型']==orderAction),'时间'].values[0]
        delivery['通知时间'] =orderTime.iloc[0,0]
        delivery['可以取单时间'] =orderTime.iloc[0,1]
        delivery['承诺送达时间'] = orderTime.iloc[0,2]
        delivery['骑手上个动作时间'] = last_action['时间']
        delivery['骑手经度'] = last_action['经度']
        delivery['骑手纬度'] = last_action['纬度']
        delivery['骑手距离'] = distance_wave.loc[(distance_wave['源订单号']==last_action['订单号']) & (distance_wave['源类型']==last_action['动作类型'])&(distance_wave['目标订单号']==orderID)&(distance_wave['目标类型']==orderAction),'距离'].values[0]
        delivery['骑手等级'] =shuxing.iloc[0,0]
        delivery['骑手速度'] =shuxing.iloc[0,1]
        delivery['最大负荷量'] = shuxing.iloc[0,2]
    
    
        
        pickupBig = pd.concat([pickupBig,pickup])
      
        deliveryBig =pd.concat([deliveryBig,delivery])
      
        
    pickupBig=pickupBig.reindex(columns=['订单号','动作','动作时间','通知时间','可以取单时间','承诺送达时间','骑手上个动作时间','骑手经度','骑手纬度','骑手距离','骑手等级','骑手速度','最大负荷量','取单号','动作类型','时间','经度','纬度','距离','理论时间','订单分配时间','订单可取时间','订单承诺送达时间','天气'])
    deliveryBig=deliveryBig.reindex(columns=['订单号','动作','动作时间','通知时间','可以取单时间','承诺送达时间','骑手上个动作时间','骑手经度','骑手纬度','骑手距离','骑手等级','骑手速度','最大负荷量','送单号','动作类型','时间','经度','纬度','距离','理论时间','订单分配时间','订单可取时间','订单承诺送达时间','天气'])    
    return pickupBig,deliveryBig
        
        
    

In [24]:

def GetPickDelivery(n):
 
    # 以一天的数据为单位进行处理
    action = train_action_list[n]
    courier = train_courier_list[n]
    distance = train_distance_list[n]
    order = train_order_list[n]
    #print(action)
    # 修改列名
    action.rename(columns=action_columns_name_dict,inplace=True)
    courier.rename(columns=courier_columns_name_dict,inplace=True)
    distance.rename(columns=distance_columns_name_dict,inplace =True)
    order.rename(columns=order_columns_name_dict,inplace =True)

    # 组合订单3个动作的表
    order_part = order[['骑手ID','波次','订单号','通知时间']]
    order_part['动作类型'] ='ASSIGN'
    order_part.rename(columns={'通知时间':'时间'},inplace = True)
    action_part = action[['骑手ID','波次','订单号','动作类型','时间']]
    order_part = order_part[['骑手ID','波次','订单号','动作类型','时间']]
    action_all = pd.concat([action_part,order_part]) # 纵向合并成一个表
    action_all_sort = action_all.sort_values(by =['骑手ID','波次','时间']) # 排序，将骑手的一个波次合并在一起
    #print(action_all)
    # 补充动作表对应的地点经纬度
    distance_part = distance[['骑手ID','波次','源订单号','源类型','源经度','源纬度']]
    distance_part.rename(columns={'源订单号':'订单号','源类型':'动作类型','源经度':'经度','源纬度':'纬度'},inplace = True)
    distance_part1 = distance_part.drop_duplicates() # 删除重复行
    #print(distance_part1)
    action_big = pd.merge(left=action_all_sort,right=distance_part1,left_on=['骑手ID','波次','订单号','动作类型'],right_on=['骑手ID','波次','订单号','动作类型'],how ='inner')
    #print(action_big)
    # 生成最后总表初始表
    data1 = order[['订单号','骑手ID','波次','天气']]
    data2 = order[['订单号','骑手ID','波次','天气']]
    columnsName =['动作','动作时间','理论完成时间','可取单时间','承诺送达时间','距离','取单号','商家经度','商家纬度','商家距离','实际取单时间','理论取单时间','取单可取时间','取单承诺送达时间','送单号','客户经度','客户纬度','客户距离','实际送单时间','理论送单时间','送单可取时间','送单承诺送达时间','骑手上个动作时间','骑手经度','骑手纬度','骑手等级','骑手速度','最大负荷量','当前已取单数']
    for st in columnsName:
        if st =='动作':
            data1.loc[:,st] ='PICKUP'
            data2.loc[:,st] = 'DELIVERY'
        else:
            data1.loc[:,st] =0
            data2.loc[:,st] =0
    data = pd.concat([data1,data2])
    dataSort =data.sort_values(by=['骑手ID','波次','订单号']) # 让一个骑手的波次聚在一块
    # 修改索引，使其从0递增
    indexlist = list(range(len(dataSort)))
    dataSort.index = indexlist
    orderSort = dataSort.loc[:,['订单号','动作','骑手ID','波次']]
    pickupOrder,deliveryOrder = GetAllOptionOrder(orderSort,action_big,order,distance,courier) # 获取每个订单动作对应的后续可选取单和送单总表
    return pickupOrder,deliveryOrder

In [30]:
# 遍历进行数据处理
picklist=list() # 存放取单
deliverylist =list() # 存放送单
for n in range(len(train_action_list)):
    pick,delivery = GetPickDelivery(n)
    picklist.append(pick)
    deliverylist.append(delivery)

In [26]:
pickdf = pd.concat(picklist)
deliverydf = pd.concat(deliverylist)

In [29]:
# 保存数据
pickdf.to_csv('../user_data/tmp_data/pickAll.txt',index=False,encoding='utf_8')
deliverydf.to_csv('../user_data/tmp_data/deliveryAll.txt',index =False,encoding='utf_8')